In [159]:
import sqlalchemy as sa
import pandas as pd
import requests, json

## Get data frome API

In [160]:
# Data Source: 	https://data.ontario.ca/api/3/action/datastore_search?resource_id=775ca815-5028-4e9b-9dd4-6975ff1be021&limit=5
api_url = 'https://data.ontario.ca/api/3/action/datastore_search?resource_id=775ca815-5028-4e9b-9dd4-6975ff1be021&limit=10000'
print(api_url)   

https://data.ontario.ca/api/3/action/datastore_search?resource_id=775ca815-5028-4e9b-9dd4-6975ff1be021&limit=10000


In [161]:
#Pull the data
data = requests.get(api_url)
data = data.json()

In [162]:
record = data['result']['records']
df = pd.DataFrame(record)
df

,_id,Date,Agegroup,At least one dose_cumulative,Second_dose_cumulative,fully_vaccinated_cumulative,third_dose_cumulative,Total population,Percent_at_least_one_dose,Percent_fully_vaccinated,Percent_3doses
0,1,2020-12-16T00:00:00,12-17yrs,0,0,0,0,961857,.0000,.0000,.0000
1,2,2020-12-16T00:00:00,18-29yrs,37,0,0,0,2454802,.0000,.0000,.0000
2,3,2020-12-16T00:00:00,30-39yrs,84,0,0,0,2077137,.0000,.0000,.0000
3,4,2020-12-16T00:00:00,40-49yrs,98,0,0,0,1855340,.0001,.0000,.0000
4,5,2020-12-16T00:00:00,50-59yrs,148,0,0,0,2019276,.0001,.0000,.0000
...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,2023-03-18T00:00:00,Adults_18plus,11284727,0,11013111,7418717,12076175,.9345,.9120,.6143
9996,9997,2023-03-18T00:00:00,Ontario_12plus,12086889,0,11771383,7624418,13038032,.9270,.9028,.5848
9997,9998,2023-03-18T00:00:00,Ontario_5plus,12626630,0,12189689,7701596,14113455,.8947,.8637,.5457
9998,9999,2023-03-18T00:00:00,Undisclosed_or_missing,1586,0,427,39,0,.0000,.0000,.0000


In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   _id                           10000 non-null  int64 
 1   Date                          10000 non-null  object
 2   Agegroup                      10000 non-null  object
 3   At least one dose_cumulative  10000 non-null  object
 4   Second_dose_cumulative        10000 non-null  object
 5   fully_vaccinated_cumulative   10000 non-null  object
 6   third_dose_cumulative         10000 non-null  object
 7   Total population              10000 non-null  object
 8   Percent_at_least_one_dose     10000 non-null  object
 9   Percent_fully_vaccinated      10000 non-null  object
 10  Percent_3doses                10000 non-null  object
dtypes: int64(1), object(10)
memory usage: 859.5+ KB


In [164]:
df.head(40)

,_id,Date,Agegroup,At least one dose_cumulative,Second_dose_cumulative,fully_vaccinated_cumulative,third_dose_cumulative,Total population,Percent_at_least_one_dose,Percent_fully_vaccinated,Percent_3doses
0,1,2020-12-16T00:00:00,12-17yrs,0,0,0,0,961857,.0000,.0000,.0000
1,2,2020-12-16T00:00:00,18-29yrs,37,0,0,0,2454802,.0000,.0000,.0000
2,3,2020-12-16T00:00:00,30-39yrs,84,0,0,0,2077137,.0000,.0000,.0000
3,4,2020-12-16T00:00:00,40-49yrs,98,0,0,0,1855340,.0001,.0000,.0000
4,5,2020-12-16T00:00:00,50-59yrs,148,0,0,0,2019276,.0001,.0000,.0000
5,6,2020-12-16T00:00:00,60-69yrs,78,0,0,0,1813493,.0000,.0000,.0000
6,7,2020-12-16T00:00:00,70-79yrs,17,0,0,0,1181151,.0000,.0000,.0000
7,8,2020-12-16T00:00:00,80+,4,0,0,0,674976,.0000,.0000,.0000
8,9,2020-12-16T00:00:00,Adults_18plus,466,0,0,0,12076175,.0000,.0000,.0000
9,10,2020-12-16T00:00:00,Ontario_12plus,466,0,0,0,13038032,.0000,.0000,.0000


# Feature Engineering 

Reported Date (reported_date)

Age Group

New Doses on fully vaccinated 

New Doses at least one dose

In [165]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   _id                           10000 non-null  int64 
 1   Date                          10000 non-null  object
 2   Agegroup                      10000 non-null  object
 3   At least one dose_cumulative  10000 non-null  object
 4   Second_dose_cumulative        10000 non-null  object
 5   fully_vaccinated_cumulative   10000 non-null  object
 6   third_dose_cumulative         10000 non-null  object
 7   Total population              10000 non-null  object
 8   Percent_at_least_one_dose     10000 non-null  object
 9   Percent_fully_vaccinated      10000 non-null  object
 10  Percent_3doses                10000 non-null  object
dtypes: int64(1), object(10)
memory usage: 859.5+ KB


In [166]:
# convert the fully_vaccinated_cumulative column to integer
df['fully_vaccinated_cumulative'] = df['fully_vaccinated_cumulative'].astype('int64')
df['At least one dose_cumulative'] = df['At least one dose_cumulative'].astype('int64')

# Caculate the new_does for each age group
#new_does_fully_vaccinated
df['new_does_fully_vaccinated']= df['fully_vaccinated_cumulative'].diff(periods=11)
#new_dose_least_one_vaccinated
df['new_dose_least_one_vaccinated']= df['At least one dose_cumulative'].diff(periods=11)

In [167]:
df['reported_date'] = pd.to_datetime(df['Date'])

In [168]:
df.drop('Date', inplace=True, axis=1)

In [169]:
df.rename(columns={"Agegroup": "age_group"},inplace= True)
df

,_id,age_group,At least one dose_cumulative,Second_dose_cumulative,fully_vaccinated_cumulative,third_dose_cumulative,Total population,Percent_at_least_one_dose,Percent_fully_vaccinated,Percent_3doses,new_does_fully_vaccinated,new_dose_least_one_vaccinated,reported_date
0,1,12-17yrs,0,0,0,0,961857,.0000,.0000,.0000,NaN,NaN,2020-12-16
1,2,18-29yrs,37,0,0,0,2454802,.0000,.0000,.0000,NaN,NaN,2020-12-16
2,3,30-39yrs,84,0,0,0,2077137,.0000,.0000,.0000,NaN,NaN,2020-12-16
3,4,40-49yrs,98,0,0,0,1855340,.0001,.0000,.0000,NaN,NaN,2020-12-16
4,5,50-59yrs,148,0,0,0,2019276,.0001,.0000,.0000,NaN,NaN,2020-12-16
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,Adults_18plus,11284727,0,11013111,7418717,12076175,.9345,.9120,.6143,-1176606.0,-1341953.0,2023-03-18
9996,9997,Ontario_12plus,12086889,0,11771383,7624418,13038032,.9270,.9028,.5848,11770956.0,12085303.0,2023-03-18
9997,9998,Ontario_5plus,12626630,0,12189689,7701596,14113455,.8947,.8637,.5457,11771383.0,12086889.0,2023-03-18
9998,9999,Undisclosed_or_missing,1586,0,427,39,0,.0000,.0000,.0000,-757845.0,-800576.0,2023-03-18


In [170]:
variables = ['reported_date','age_group','At least one dose_cumulative','new_dose_least_one_vaccinated','new_does_fully_vaccinated','fully_vaccinated_cumulative']
df = df[variables]
df[9900:9940]

,reported_date,age_group,At least one dose_cumulative,new_dose_least_one_vaccinated,new_does_fully_vaccinated,fully_vaccinated_cumulative
9900,2023-03-11,50-59yrs,1829273,636059.0,618546.0,1796288
9901,2023-03-11,60-69yrs,1784826,1108487.0,1088325.0,1755804
9902,2023-03-11,70-79yrs,1193187,-10092297.0,-9836230.0,1177712
9903,2023-03-11,80+,676246,-11411347.0,-11104758.0,667386
9904,2023-03-11,Adults_18plus,11285424,-1341776.0,-1176231.0,11013847
9905,2023-03-11,Ontario_12plus,12087540,12085954.0,11771634.0,11772061
9906,2023-03-11,Ontario_5plus,12627161,12087540.0,11772061.0,12190033
9907,2023-03-11,Undisclosed_or_missing,1586,-800530.0,-757787.0,427
9908,2023-03-12,05-11yrs,539640,-1655343.0,-1690780.0,418041
9909,2023-03-12,12-17yrs,802119,-1097926.0,-1081442.0,758225


## Data Oberservation

From the obeservation we could see, the 'new_dose_least_one_vaccinated' and 'new_does_fully_vaccinated' started to become an negative number which is not reasonable. By observe the 'At least one dose_cumulative','fully_vaccinated_cumulative' and compare it to the 'new_dose_least_one_vaccinated','new_does_fully_vaccinated' we find out the cumulative start decrease at some point which means this dataset might need advanced exploration and data cleaning to make it correct.

I suggest next step, we should check where did the cumulative start to decrease. It might cause by missing data or typo,etc.

In [171]:
variables = ['reported_date','age_group','new_dose_least_one_vaccinated','new_does_fully_vaccinated']
df = df[variables]
df.head(50)

,reported_date,age_group,new_dose_least_one_vaccinated,new_does_fully_vaccinated
0,2020-12-16,12-17yrs,NaN,NaN
1,2020-12-16,18-29yrs,NaN,NaN
2,2020-12-16,30-39yrs,NaN,NaN
3,2020-12-16,40-49yrs,NaN,NaN
4,2020-12-16,50-59yrs,NaN,NaN
5,2020-12-16,60-69yrs,NaN,NaN
6,2020-12-16,70-79yrs,NaN,NaN
7,2020-12-16,80+,NaN,NaN
8,2020-12-16,Adults_18plus,NaN,NaN
9,2020-12-16,Ontario_12plus,NaN,NaN


# Ingest the data into PostgreSQL server

In [172]:
my_db_secret = {
    'drivername': 'postgresql+psycopg2', 
    'host': 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port': '5432', 
    'username': 'shawnmd', 
    'password': '2023!Schulich',
    'database': 'shawnmd_db'
}

In [173]:
my_db_url = sa.engine.URL.create(
    drivername = my_db_secret['drivername'],
    username = my_db_secret['username'],
    password = my_db_secret['password'],
    host = my_db_secret['host'],
    port = my_db_secret['port'],
    database = my_db_secret['database']
)

In [174]:
my_db_engine = sa.create_engine(my_db_url)

In [175]:
df.to_sql(
    name = 'covid_on_vaccine_data',
    schema = 'final_exam',
    con = my_db_engine,
    if_exists = 'replace',
    index=False,
    method='multi',
    dtype= {
        'reported_date' :sa.types.CHAR(10000),
        'age_group' :sa.types.VARCHAR(100),
         'new_dose_least_one_vaccinated': sa.types.INTEGER,
           'new_does_fully_vaccinated': sa.types.INTEGER,
    }   
)

10000